Transform trajectories with bid base action

In [ ]:
from PIL import  Image
import json, os
from browsergym.workarena.tasks import form
from browsergym.workarena.tasks import list as wa_list

import pickle
data = pickle.load(open("trace_profiling/task_traces.pkl", "rb"))

tasks = [
    wa_list.FilterAssetListTask,
    wa_list.FilterChangeRequestListTask,
    wa_list.FilterHardwareListTask,
    wa_list.FilterServiceCatalogItemListTask,
    wa_list.FilterUserListTask,
    wa_list.SortAssetListTask,
    wa_list.SortChangeRequestListTask,
    wa_list.SortHardwareListTask,
    wa_list.SortIncidentListTask,
    wa_list.SortServiceCatalogItemListTask,
    wa_list.SortUserListTask,
    form.CreateChangeRequestTask,
    form.CreateIncidentTask,
    form.CreateHardwareAssetTask,
    form.CreateProblemTask,
    form.CreateUserTask
]

for task in tasks:
    for i, traj in enumerate(data[task]):
        base_dir = os.path.join("trajectories", task.__name__, str(i))
        os.makedirs(base_dir, exist_ok=True)
        traj_json = []
        prev_action = []
        goal = traj[0]["obs"]["goal"].strip()
        for i in range(len(traj)):
            image = Image.fromarray(traj[i]["obs"]["screenshot"])
            image.save(os.path.join(base_dir, f"{i}.png"))
            extra = traj[i]["obs"]["extra_element_properties"]
            action = traj[i]["action"]
            if action == "click":
                bid = traj[i]["bid"]
                bbox = extra[bid]["bbox"]
                x = int(bbox[0] + bbox[2] / 2)
                y = int(bbox[1] + bbox[3] / 2)
                action = f"click({x}, {y})"
            elif action == "fill":
                bid = traj[i]["bid"]
                bbox = extra[bid]["bbox"]
                x = int(bbox[0] + bbox[2] / 2)
                y = int(bbox[1] + bbox[3] / 2)
                args = traj[i]["args"]
                assert len(args) == 1
                action = f"click({x}, {y})\ntype(\"{args[0]}\")"
            elif action == "select_option":
                bid = traj[i]["bid"]
                bbox = extra[bid]["bbox"]
                x = int(bbox[0] + bbox[2] / 2)
                y = int(bbox[1] + bbox[3] / 2)
                args = traj[i]["args"]
                assert len(args) == 1
                action = f"select_option({x}, {y}, options=\"{args[0]}\")"
            elif action == "press":
                args = traj[i]["args"]
                assert len(args) == 1
                action = f"press(\"{args[0]}\")"
            elif action == "set_checked":
                bid = traj[i]["bid"]
                bbox = extra[bid]["bbox"]
                x = int(bbox[0] + bbox[2] / 2)
                y = int(bbox[1] + bbox[3] / 2)
                action = f"click({x}, {y})"
            else:
                raise ValueError(f"Unknown action: {action}")
            
            if len(prev_action) > 0:
                prev_action_str = "\n".join(prev_action)
            else:
                prev_action_str = "None"

            prev_action.append(action)

            step_json = {
                "image": os.path.join(base_dir, f"{i}.png"),
                "conversations": [
                    {
                        "from": "system",
                        "value": "You are a GUI agent."
                    },
                    {
                        "from": "human",
                        "value": f"<image>\nPlease generate the next move according to the UI screenshot, instruction and previous actions.\n\nInstruction:{goal}\n\nPrevious actions:\n{prev_action_str}"
                    },
                    {
                        "from": "gpt",
                        "value": action
                    }
                ]
            }
            traj_json.append(step_json)
        json.dump(traj_json, open(os.path.join(base_dir, "traj.json"), "w"), indent=2)



In [4]:

import pickle
files = ["trace_profiling/FormTask1_20.pkl", "trace_profiling/FormTask2_20.pkl", "trace_profiling/FormTask3_20.pkl", "trace_profiling/SortTask1_20.pkl", "trace_profiling/SortTask2_20.pkl", "trace_profiling/ServiceTask20.pkl", "trace_profiling/FilterTask20.pkl", "trace_profiling/FilterTask20first3.pkl"]

data = pickle.load(open(files[2], "rb"))

In [5]:
from browsergym.workarena.tasks import form
step = data[form.CreateChangeRequestTask][0][8]
step.keys()

dict_keys(['obs', 'action', 'args', 'kwargs', 'bid', 'time'])

In [15]:

from browsergym.workarena.tasks import form, service_catalog, dashboard, knowledge
bid = data[form.CreateIncidentTask][0][0]['bid']
data[form.CreateIncidentTask][0][0]['obs']['extra_element_properties'][bid]

{'visibility': 1.0,
 'bbox': [227.328125, 377, 861.15625, 32],
 'clickable': True,
 'set_of_marks': True}

Transform trajectories with coords based action

In [3]:
from PIL import Image, ImageDraw
import json, os
import pickle
from browsergym.workarena.tasks import form, service_catalog, dashboard, knowledge
from browsergym.workarena.tasks import list as wa_list

files = [f"trace_profiling/SFTData{i}.pkl" for i in range(10)]
tasks = [
    wa_list.FilterAssetListTask,
    wa_list.FilterChangeRequestListTask,
    wa_list.FilterHardwareListTask,
    wa_list.FilterServiceCatalogItemListTask,
    wa_list.FilterUserListTask,
    wa_list.SortAssetListTask,
    wa_list.SortChangeRequestListTask,
    wa_list.SortHardwareListTask,
    wa_list.SortIncidentListTask,
    wa_list.SortServiceCatalogItemListTask,
    wa_list.SortUserListTask,
    form.CreateChangeRequestTask,
    form.CreateIncidentTask,
    form.CreateHardwareAssetTask,
    form.CreateProblemTask,
    form.CreateUserTask,
    service_catalog.OrderDeveloperLaptopTask,
    service_catalog.OrderIpadMiniTask,
    service_catalog.OrderIpadProTask,
    service_catalog.OrderSalesLaptopTask,
    service_catalog.OrderStandardLaptopTask,
    service_catalog.OrderAppleWatchTask,
    service_catalog.OrderAppleMacBookPro15Task,
    service_catalog.OrderDevelopmentLaptopPCTask,
    service_catalog.OrderLoanerLaptopTask,
]

def draw_circle_on_image(x, y, image: Image.Image, radius=5):
    """
    Draw a circle on the image at the specified (x, y) coordinates.
    """
    draw = ImageDraw.Draw(image)
    left = x - radius
    top = y - radius
    right = x + radius
    bottom = y + radius
    draw.ellipse([left, top, right, bottom], fill="red", outline="red")
    return image

def draw_bbox_on_image(bbox, image: Image.Image):
    """
    Draw a circle on the image at the specified (x, y) coordinates.
    """
    draw = ImageDraw.Draw(image)
    x, y, w, h = bbox
    draw.rectangle([x, y, x + w, y + h], outline='red', width=5)
    return image

for file in files:
    data = pickle.load(open(file, "rb"))
    for task in data.keys():
        if task not in data.keys():
            continue
        dir = os.path.join("trajectories_coords", task.__name__)
        os.makedirs(dir, exist_ok=True)
        num = len(os.listdir(dir))
        for traj_index, traj in enumerate(data[task]):
            base_dir = os.path.join("trajectories_coords", task.__name__, str(num + traj_index))
            os.makedirs(base_dir, exist_ok=True)
            traj_json = []
            prev_action = []
            goal = traj[0]["obs"]["goal"].strip()
            for i in range(len(traj)):
                bid = traj[i]['bid']
                try:
                    extra = traj[i]["obs"]["extra_element_properties"][bid]
                except Exception as e:
                    if bid == 'mouse' and traj[i]["action"] == "click":
                        prev_bid = traj[i-1]['bid']
                        prev_extra = traj[i-1]["obs"]["extra_element_properties"]
                        x, _, width, _ = prev_extra[prev_bid]['bbox']
                        _, y = traj[i]["args"]
                        height = 22
                        y -= height / 2
                        extra = {
                            "visibility": True,
                            "bbox": [x, y, width, height]
                        }
                    else:
                        extra = {
                            "visibility": False,
                            "bbox": None
                        }
                if extra['visibility']:
                    image = Image.fromarray(traj[i]["obs"]["screenshot"])
                    image = draw_bbox_on_image(extra['bbox'], image)
                    image.save(os.path.join(base_dir, f"{i}_bbox.png"))                
                image = Image.fromarray(traj[i]["obs"]["screenshot"])
                image.save(os.path.join(base_dir, f"{i}.png"))
                action = traj[i]["action"]
                if action == "click":
                    x, y = traj[i]["args"]
                    action = f"click({x}, {y})"
                    image = draw_circle_on_image(x, y, image)
                    image.save(os.path.join(base_dir, f"{i}_click.png"))
                elif action == "type":
                    args = traj[i]["args"]
                    assert len(args) == 1
                    if args[0] == "":
                        action = ""
                    else:
                        action = f"type(\"{args[0]}\")"
                elif action == "hotkey":
                    args = traj[i]["args"]
                    assert len(args) == 1
                    action = f"hotkey(\"{args[0]}\")"
                elif action == "wheel":
                    x, y = traj[i]["args"]
                    action = f"scroll({x}, {y})"
                elif action == "finish":
                    action = "finish()"
                elif action == "empty":
                    action = ""
                else:
                    raise ValueError(f"Unknown action: {action}")
                
                if len(prev_action) > 0:
                    prev_action_str = "\n".join(prev_action)
                else:
                    prev_action_str = "None"

                if action == "":
                    print("found empty")
                else:
                    prev_action.append(action)

                step_json = {
                    "image": os.path.join(base_dir, f"{i}.png"),
                    "bbox": extra['bbox'] if extra['visibility'] else None,
                    "conversations": [
                        {
                            "from": "system",
                            "value": "You are a GUI agent."
                        },
                        {
                            "from": "human",
                            "value": f"<image>\nPlease generate the next move according to the UI screenshot, instruction and previous actions.\n\nInstruction:{goal}\n\nPrevious actions:\n{prev_action_str}"
                        },
                        {
                            "from": "gpt",
                            "value": action
                        }
                    ]
                }
                traj_json.append(step_json)
            json.dump(traj_json, open(os.path.join(base_dir, "traj.json"), "w"), indent=2)
    del data
    
# os.system("rclone copy --progress ./trajectories_coords volces-tos:tos-bjml-ma4agi/mabasic/chenyinghao/workarena/trajectories_coords_sft ")



found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
found empty
foun

['trace_profiling/grpoData.pkl']